In [1]:
import numpy as np
import geopandas as gpd
import shapely
import matplotlib.pyplot as plt

In [2]:
# Load NYC borough shapefile
path = '../data/shapefile/nyc_boroughs.shp'
nyc = gpd.read_file(path)
# Transform to WGS84 (EPSG:4326) to match crime data coordinates
nyc = nyc.to_crs("EPSG:4326")
nyc = nyc.dissolve()

# Load data to get dimensions
train_data_temp = np.load('../data/train_data.npy')
n_y_cells, n_x_cells = train_data_temp.shape[1], train_data_temp.shape[2]

# Create mask using shapefile bounds (like Chicago version)
xmin, ymin, xmax, ymax = nyc.total_bounds
x_cell_size = (xmax - xmin) / n_x_cells
y_cell_size = (ymax - ymin) / n_y_cells
mask = np.ones((n_y_cells, n_x_cells))
nyc_geom = nyc.geometry.values[0]

x_arange = np.arange(xmin, xmax+x_cell_size, x_cell_size)
y_arange = np.arange(ymin, ymax+y_cell_size, y_cell_size)
for i, y0 in zip(range(n_y_cells-1, -1, -1), y_arange):
    for j, x0 in zip(range(n_x_cells), x_arange):
        x1 = x0 + x_cell_size
        y1 = y0 + y_cell_size
        box = shapely.geometry.box(x0, y0, x1, y1)
        if not nyc_geom.intersects(box):
            mask[i,j] = 0

print(f"Grid dimensions for NYC: {n_y_cells} x {n_x_cells}")
print(f"Cells inside NYC: {np.sum(mask)}")

Grid dimensions for NYC: 67 x 50
Cells inside NYC: 1886.0


In [3]:
res_Cr20 = np.load('../data/final_pred_Cr_20epoch.npy')
res_Cr_relu = np.load('../data/final_pred_Cr_relu.npy')
res_Cr_linear = np.load('../data/final_pred_Cr_linear.npy')
res_Cr_Sev_linear = np.load('../data/final_pred_Cr+Sev_linear.npy')
# res_Cr_Sev_Sea_linear = np.load('../data/final_pred_Cr_Sev_Sea_linear.npy')

res_Cr = np.load('../data/final_pred_Cr.npy')
res_Cr_Sev = np.load('../data/final_pred_Cr+Sev.npy')
res_Cr_Sev_Sea = np.load('../data/final_pred_Cr+Sev+Sea.npy')

res_Cr_Homo = np.load('../data/final_pred_ConvLSTM_Cr.npy')[:, :, :, 0]

ground_truth = np.load('../data/test_data.npy')[7:]

FileNotFoundError: [Errno 2] No such file or directory: '../data/final_pred_Cr_20epoch.npy'

In [ ]:
res_Cr20 /= mask
res_Cr_relu /= mask
res_Cr_linear /= mask
res_Cr_Sev_linear /= mask
# res_Cr_Sev_Sea_linear /= mask

res_Cr /= mask
res_Cr_Sev /= mask
res_Cr_Sev_Sea /= mask

res_Cr_Homo /= mask

ground_truth /= mask

C:\Users\lcdew\AppData\Local\Temp\ipykernel_17380\3523072320.py:1: RuntimeWarning: invalid value encountered in divide
  res_Cr20 /= mask
C:\Users\lcdew\AppData\Local\Temp\ipykernel_17380\3523072320.py:2: RuntimeWarning: invalid value encountered in divide
  res_Cr_relu /= mask
C:\Users\lcdew\AppData\Local\Temp\ipykernel_17380\3523072320.py:3: RuntimeWarning: invalid value encountered in divide
  res_Cr_linear /= mask
C:\Users\lcdew\AppData\Local\Temp\ipykernel_17380\3523072320.py:4: RuntimeWarning: invalid value encountered in divide
  res_Cr_Sev_linear /= mask
C:\Users\lcdew\AppData\Local\Temp\ipykernel_17380\3523072320.py:7: RuntimeWarning: invalid value encountered in divide
  res_Cr /= mask
C:\Users\lcdew\AppData\Local\Temp\ipykernel_17380\3523072320.py:8: RuntimeWarning: invalid value encountered in divide
  res_Cr_Sev /= mask
C:\Users\lcdew\AppData\Local\Temp\ipykernel_17380\3523072320.py:9: RuntimeWarning: invalid value encountered in divide
  res_Cr_Sev_Sea /= mask
C:\Users\lc

In [ ]:
mse = np.nanmean(np.square(np.subtract(ground_truth, res_Cr20)))
print(f'Hetero-ConvLSTM - MSE: {mse.mean():.4f}')
print(f'Hetero-ConvLSTM - RMSE: {np.sqrt(mse).mean():.4f}')

Hetero-ConvLSTM - MSE: 0.6383
Hetero-ConvLSTM - RMSE: 0.7989


In [ ]:
mse = np.nanmean(np.square(np.subtract(ground_truth, res_Cr_relu)))
print(f'Hetero-ConvLSTM - MSE: {mse.mean():.4f}')
print(f'Hetero-ConvLSTM - RMSE: {np.sqrt(mse).mean():.4f}')

Hetero-ConvLSTM - MSE: 0.5379
Hetero-ConvLSTM - RMSE: 0.7334


In [ ]:
res_Cr_linear_clip = np.clip(res_Cr_linear, 0, None)
mse = np.nanmean(np.square(np.subtract(ground_truth, res_Cr_linear_clip)))
print(f'Hetero-ConvLSTM - MSE: {mse.mean():.4f}')
print(f'Hetero-ConvLSTM - RMSE: {np.sqrt(mse).mean():.4f}')

Hetero-ConvLSTM - MSE: 0.5369
Hetero-ConvLSTM - RMSE: 0.7327


In [ ]:
res_Cr_Sev_linear_clip = np.clip(res_Cr_Sev_linear, 0, None)
mse = np.nanmean(np.square(np.subtract(ground_truth, res_Cr_Sev_linear_clip)))
print(f'Hetero-ConvLSTM - MSE: {mse.mean():.4f}')
print(f'Hetero-ConvLSTM - RMSE: {np.sqrt(mse).mean():.4f}')

Hetero-ConvLSTM - MSE: 0.5404
Hetero-ConvLSTM - RMSE: 0.7351


In [ ]:
res_Cr_clip = np.clip(res_Cr, 0, None)
mse = np.nanmean(np.square(np.subtract(ground_truth, res_Cr_clip)))
print(f'Hetero-ConvLSTM - MSE: {mse.mean():.4f}')
print(f'Hetero-ConvLSTM - RMSE: {np.sqrt(mse).mean():.4f}')

Hetero-ConvLSTM - MSE: 0.6123
Hetero-ConvLSTM - RMSE: 0.7825


In [ ]:
res_Cr_Sev_clip = np.clip(res_Cr_Sev, 0, None)
mse = np.nanmean(np.square(np.subtract(ground_truth, res_Cr_Sev_clip)))
print(f'Hetero-ConvLSTM - MSE: {mse.mean():.4f}')
print(f'Hetero-ConvLSTM - RMSE: {np.sqrt(mse).mean():.4f}')

Hetero-ConvLSTM - MSE: 0.6129
Hetero-ConvLSTM - RMSE: 0.7829


In [ ]:
res_Cr_Sev_Sea_clip = np.clip(res_Cr_Sev_Sea, 0, None)
mse = np.nanmean(np.square(np.subtract(ground_truth, res_Cr_Sev_Sea_clip)))
print(f'Hetero-ConvLSTM - MSE: {mse.mean():.4f}')
print(f'Hetero-ConvLSTM - RMSE: {np.sqrt(mse).mean():.4f}')

Hetero-ConvLSTM - MSE: 0.6273
Hetero-ConvLSTM - RMSE: 0.7921


In [ ]:
res_Cr_Homo_clip = np.clip(res_Cr_Homo, 0, None)
mse = np.nanmean(np.square(np.subtract(ground_truth, res_Cr_Homo_clip)))
print(f'Hetero-ConvLSTM - MSE: {mse.mean():.4f}')
print(f'Hetero-ConvLSTM - RMSE: {np.sqrt(mse).mean():.4f}')

Hetero-ConvLSTM - MSE: 0.6127
Hetero-ConvLSTM - RMSE: 0.7828
